# Redis Short-Term Memory Demo
This notebook demonstrates how to use the Redis-based short-term memory system for storing and retrieving conversation context.

In [ ]:
# Install required packages if not already installed
!pip install redis

In [ ]:
import os
import asyncio
import json
from datetime import datetime, timezone
from pathlib import Path
import sys

# Add the project root to the Python path
project_root = str(Path.cwd().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the ShortTermMemory class
from app.services.memory.short_term import ShortTermMemory
from app.config import Settings
from app.core.logging import setup_logging

# Set up logging
setup_logging()

In [ ]:
# Create test settings
settings = Settings(
    redis_host="localhost",
    redis_port=6379,
    # Enable only short-term memory for this demo
    memory_enabled=True,
    enable_short_term_memory=True,
    enable_semantic_memory=False,
    enable_episodic_memory=False,
    enable_procedural_memory=False,
    # Disable other services for this demo
    enable_sql=False,
    enable_mongo=False,
    enable_neo4j=False,
    enable_chroma=False
)

# Initialize the short-term memory with a 1-hour TTL
memory = ShortTermMemory(settings=settings, ttl=3600)

In [ ]:
async def demo_short_term_memory():
    # Generate a unique conversation ID
    conversation_id = f"conv_{datetime.now(timezone.utc).timestamp()}"
    print(f"Starting demo with conversation ID: {conversation_id}")
    
    # Store a message in the conversation
    message1 = {
        "role": "user",
        "content": "Hello, how are you?",
        "timestamp": datetime.now(timezone.utc).isoformat()
    }
    
    message1_key = f"{conversation_id}:msg1"
    await memory.store(
        key=message1_key,
        content=message1,
        metadata={"source": "demo"},
        conversation_id=conversation_id
    )
    print(f"Stored message 1 with key: {message1_key}")
    
    # Store another message in the same conversation
    message2 = {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I help you today?",
        "timestamp": datetime.now(timezone.utc).isoformat()
    }
    
    message2_key = f"{conversation_id}:msg2"
    await memory.store(
        key=message2_key,
        content=message2,
        metadata={"source": "demo"},
        conversation_id=conversation_id
    )
    print(f"Stored message 2 with key: {message2_key}")
    
    # Retrieve the conversation history
    print("\nRetrieving conversation history...")
    conversation = []
    for key in [message1_key, message2_key]:
        message = await memory.retrieve(key)
        if message:
            conversation.append(f"{message['role'].title()}: {message['content']}")
    
    for message in conversation:
        print(f"- {message}")
    
    # Search for messages in the conversation
    print("\nSearching for messages containing 'help'...")
    search_results = await memory.search(
        query="help",
        conversation_id=conversation_id,
        limit=5
    )
    
    if search_results:
        print("Found matching messages:")
        for result in search_results:
            content = result.get("content", {})
            if isinstance(content, dict):
                role = content.get("role", "unknown")
                text = content.get("content", "No content")
                print(f"- {role.title()}: {text}")
            else:
                print(f"- {content}")
    else:
        print("No matching messages found.")
    
    # Clean up
    print("\nCleaning up...")
    await memory.delete(message1_key)
    await memory.delete(message2_key)
    
    print("\nDemo completed successfully!")

# Run the demo
await demo_short_term_memory()

## Conclusion

This notebook demonstrated how to use the Redis-based short-term memory system to:

1. Store conversation messages with metadata
2. Retrieve individual messages by key
3. Search for messages within a conversation
4. Clean up after the demo

The short-term memory system is particularly useful for maintaining conversation context in chatbot applications.